In [1]:
from art.defences.trainer import (AdversarialTrainerFBFPyTorch,
                                AdversarialTrainerMadryPGD,
                                AdversarialTrainer)

/home/sweta/scratch/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-09-21 11:34:22.284186: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-21 11:34:22.416893: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-21 11:34:22.422408: W tensorflow/stream_executor/platform/defa

In [100]:
"""
MNIST classifier testing
"""
import sys
sys.path.append("..")

from models import MNISTClassifier
from dataloader import load_mnist

import pytorch_lightning as pl

batch_size = 1000
train_dataloader, valid_dataloader, test_dataloader = load_mnist(root="/home/sweta/scratch/datasets/MNIST/", batch_size=batch_size)

model = MNISTClassifier()
trainer = pl.Trainer(max_epochs=10, gpus=1, default_root_dir="..")
# trainer.fit(model, train_dataloader)   

model = MNISTClassifier.load_from_checkpoint("../lightning_logs/version_6/checkpoints/epoch=9-step=9370.ckpt")
model.eval()
# preds = trainer.predict(model, dataloaders=test_dataloader, return_predictions=True)
# print(len(preds), preds[0].shape)
p = trainer.test(model, dataloaders=test_dataloader, verbose=False)
print(p)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Testing DataLoader 0: 100%|██████████| 10/10 [00:01<00:00,  9.78it/s]
[{'test_acc': 0.9800999760627747}]


In [101]:
import numpy as np

from art.attacks.evasion import DeepFool, ProjectedGradientDescentPyTorch
from art.estimators.classification import PyTorchClassifier
from torch import nn, optim
import torch.nn.functional as F

model.train()

x_test, y_test = next(iter(test_dataloader))
x_test, y_test = x_test.to("cuda"), y_test.to("cuda")
y_test_one_hot = F.one_hot(y_test, num_classes=10)
x_test_np, y_test_np, y_test_one_hot_np = x_test.cpu().numpy(), y_test.cpu().numpy(), y_test_one_hot.cpu().numpy()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Step 4: Create the ART classifier
classifier = PyTorchClassifier( 
    model=model,
    clip_values=(0,1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 784),
    nb_classes=10,
)

predictions = classifier.predict(x_test_np)
accuracy = np.sum(np.argmax(predictions, axis=-1) == y_test_np) / len(y_test_np)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

attack = ProjectedGradientDescentPyTorch(classifier)
x_test_adv_np = attack.generate(x=x_test_np)
predictions = classifier.predict(x_test_adv_np)
accuracy = np.sum(np.argmax(predictions, axis=-1) == y_test_np) / len(y_test_np)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 97.7%


Accuracy on adversarial test examples: 1.0999999999999999%


In [102]:
from art.data_generators import PyTorchDataGenerator

trainer = AdversarialTrainerMadryPGD(classifier)

# art_datagen = PyTorchDataGenerator(iterator=test_dataloader, size=len(test_dataloader)*batch_size, batch_size=batch_size)
# trainer.fit_generator(art_datagen)

trainer.fit(x_test_np, y_test_one_hot_np, nb_epochs=100)

Adversarial training epochs: 100%|██████████| 100/100 [00:47<00:00,  2.11it/s]


In [104]:
from art.attacks.evasion import DeepFool, FastGradientMethod

attack = FastGradientMethod(classifier)
x_test_adv_np = attack.generate(x=x_test_np)
predictions = classifier.predict(x_test_adv_np)
accuracy = np.sum(np.argmax(predictions, axis=-1) == y_test_np) / len(y_test_np)
print("Accuracy on adversarial test examples: {}%".format(accuracy * 100))

Accuracy on adversarial test examples: 12.6%


In [18]:
import numpy as np
import torch

n = np.arange(36).reshape((1,3,3,4))
a = torch.from_numpy(n)
perm = torch.LongTensor([0,2,1])

In [ ]:
a[:,:,perm,:]